In [1]:
import requests

# limited 10 calls per minute
def get_stock_news(stock_codes, n, lastId=None):
    last = ''
    if lastId:
        last = f'&last={lastId}'
    query = ''
    for symbol in stock_codes:
        query = query + 'tt:' + symbol +' '
    url = f"https://api.tickertick.com/feed?q=(and (or {query}) (or s:axios s:reuters s:bloomberg))&n={n}{last}"
#     url = f"https://api.tickertick.com/feed?q=(diff (or tt:aapl tt:goog: tt:fb tt:nflx tt:amzn tt:tsla tt:mmm tt:aos tt:abt) s:reddit)&hours_ago=1"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        # do something with the data
        stories = data['stories']
#         get_value = lambda item: item.get("description", item["title"])
#         refactor = list(map(get_value, stories))
        return stories
    else:
        print("Failed to get news data, status code:", response.status_code)
        return [{"title":"failed"}]

In [2]:
import pandas as pd
import numpy as np
import time

In [3]:
df = pd.read_csv('sp500_info.csv')

In [4]:
new_df = pd.DataFrame(df['Symbol'].copy())

In [5]:
new_df['news'] = 0

In [6]:
for i in range(len(df)):
    print(i,new_df['Symbol'][i])
    news = get_stock_news([new_df['Symbol'][i]], 100)
    if len(news) == 0:
        title_string = "None"
    else:
        title_string = ""
        for newletter in news:
            title = newletter["title"]
            if title_string != "":
                title_string = title_string+"/n"+title
            else:
                title_string = title_string+title
    new_df.loc[i, 'news'] = title_string
    if i%10 == 9:
        time.sleep(120)

0 MMM
1 AOS
2 ABT
3 ABBV
4 ABMD
5 ACN
6 ATVI
7 ADM
8 ADBE
9 AAP
10 AMD
11 AES
12 AFL
13 A
14 APD
15 AKAM
16 ALK
17 ALB
18 ARE
19 ALGN
20 ALLE
21 LNT
22 ALL
23 GOOGL
24 GOOG
25 MO
26 AMZN
27 AMCR
28 AEE
29 AAL
30 AEP
31 AXP
32 AIG
33 AMT
34 AWK
35 AMP
36 ABC
37 AME
38 AMGN
39 APH
40 ADI
41 ANSS
42 ANTM
43 AON
44 APA
45 AAPL
46 AMAT
47 APTV
48 ANET
49 AJG
50 AIZ
51 T
52 ATO
53 ADSK
54 ADP
55 AZO
56 AVB
57 AVY
58 BKR
59 BLL
60 BAC
61 BBWI
62 BAX
63 BDX
64 BRK.B
65 BBY
66 BIO
67 TECH
68 BIIB
69 BLK
70 BK
71 BA
72 BKNG
73 BWA
74 BXP
75 BSX
76 BMY
77 AVGO
78 BR
79 BRO
80 BF.B
81 CHRW
82 CDNS
83 CZR
84 CPB
85 COF
86 CAH
87 KMX
88 CCL
89 CARR
90 CTLT
91 CAT
92 CBOE
93 CBRE
94 CDW
95 CE
96 CNC
97 CNP
98 CDAY
99 CERN
100 CF
101 CRL
102 SCHW
103 CHTR
104 CVX
105 CMG
106 CB
107 CHD
108 CI
109 CINF
110 CTAS
111 CSCO
112 C
113 CFG
114 CTXS
115 CLX
116 CME
117 CMS
118 KO
119 CTSH
120 CL
121 CMCSA
122 CMA
123 CAG
124 COP
125 ED
126 STZ
127 CPRT
128 GLW
129 CTVA
130 COST
131 CTRA
132 CCI
133 CSX
134 CM

In [7]:
new_df.to_csv("simple_news.csv")